# Installing Libraries

In [ ]:
#!pip install pandas_datareader

In [ ]:
#!pip install yfinance

In [ ]:
#!pip install yfinance --upgrade --no-cache-dir


In [ ]:
#!pip install html5lib

# Importing Libraries

In [3]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from unidecode import unidecode
import re
import os
import yfinance as yf
from time import sleep
import glob
import regex as re

# Gathering Data - Brazilian Companies

### List of brazilian companies and composition of indexes - Ibovespa, IBrX100, IBrX50, IBrA

## Selenium

Used to get list of companies in brazilian's index

### Configurating WebDriver

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

### Configurating download file

In [5]:
chromeOptions = webdriver.ChromeOptions()
download_path = r'C:\Users\Pedro\OneDrive\Desktop\Ironhack\04. GitHub\stocks_project\data'
prefs = {"download.default_directory" : download_path}
chromeOptions.add_experimental_option("prefs",prefs)

In [ ]:
#Runs the webdriver
driver = webdriver.Chrome(options=chromeOptions)

### Download Index File (.csv)

In [ ]:
def get_index_stocks(index, wait=6):
    '''
        Receives the Index name, download a file that contains the index tickers, and return the name of the downloaded file
    '''

    # Chrome WebDriver opens the index webside
    url = f'https://sistemaswebb3-listados.b3.com.br/indexPage/day/{index.upper()}?language=pt-br'
    driver.get(url)
    driver.implicitly_wait(wait)

    driver.find_element(By.ID, 'segment').send_keys("Setor de Atuação")
    driver.implicitly_wait(wait)
    driver.find_element(By.LINK_TEXT, 'Download').click()
    driver.implicitly_wait(wait)

    # Set the directory
    os.chdir(r'C:\Users\Pedro\OneDrive\Desktop\Ironhack\04. GitHub\stocks_project\data')
    sleep(wait)

    # Get the .csv files from the selectec directory and sort them ascending by modification date 
    files = list(glob.glob('*csv'))
    files.sort(key=lambda x: os.path.getmtime(x), reverse=True)

    # Returns the name of the most recent file
    return files[0]

### Create the Index DataFrame

In [ ]:
def create_df(file):
    '''
        This function receives the name of the Stock Exchange index and returns a DataFrame with all companies and their respective sectors
    '''

    # Creating DataFrame
    DataFrame = pd.read_csv(file, 
                                encoding='ISO-8859-1',
                                header=1,                   # Uses line 1 as header
                                sep=';',                    
                                decimal=',',
                                thousands='.',
                                skipfooter=2,               # Removes last 2 lines
                                engine='python',
                                index_col=False)            # Does not make first column as index
    
    # Normalizing columns
    DataFrame.columns = [re.sub('[\.()]', '', re.sub(' ', '_', unidecode(columns.lower()))) for columns in DataFrame.columns]

    return DataFrame

## Creating Index's DataFrames

In [6]:
# Defining file's path
path = r'C:\Users\Pedro\OneDrive\Desktop\Ironhack\04. GitHub\stocks_project\data/'

In [ ]:
# Defining DataFrames' columns names
col_names = [
    'sector',
    'ticker',
    'name',
    'type',
    'amount',
    'percentage',
    'percentage_acum'
]

### Ibovespa

In [ ]:
ibov = create_df(get_index_stocks('ibov'))
len(ibov)

In [ ]:
ibov.columns = col_names

In [ ]:
# Save index file
ibov.to_csv(f'{path}IBOV.csv',
                    encoding='ISO-8859-1',
                    sep=';',
                    decimal='.',
                    index=False)

### IBrX100

In [ ]:
ibrx = create_df(get_index_stocks('ibxx'))
len(ibrx)

In [ ]:
ibrx.columns = col_names

In [ ]:
# Save index file
ibrx.to_csv(f'{path}IBRX100.csv',
                    encoding='ISO-8859-1',
                    sep=';',
                    decimal='.',
                    index=False)

### IBrX50

In [ ]:
ibrx50 = create_df(get_index_stocks('ibxl'))
len(ibrx50)

In [ ]:
ibrx50.columns = col_names

In [ ]:
# Save index file
ibrx50.to_csv(f'{path}IBRX50.csv',
                    encoding='ISO-8859-1',
                    sep=';',
                    decimal='.',
                    index=False)

### IBrA

In [ ]:
ibra = create_df(get_index_stocks('ibra'))
len(ibra)

In [ ]:
ibra.columns = col_names

In [ ]:
# Save index file
ibra.to_csv(f'{path}IBRA.csv',
                    encoding='ISO-8859-1',
                    sep=';',
                    decimal='.',
                    index=False)

### Transform Sector - Obsoleto?

In [ ]:
ibra = pd.read_csv(f'{path}IBRA.csv',
                    encoding='ISO-8859-1',
                    sep=';',
                    decimal='.')

In [ ]:
len(ibra)

In [ ]:
ibra

In [ ]:
ibra[['sector_aux', 'sub_sector_aux']] = ibra['sector'].str.split('/',expand=True)

In [ ]:
ibra['sector_aux'] = ibra['sector_aux'].str.strip()
ibra['sub_sector_aux'] = ibra['sub_sector_aux'].str.strip()

In [ ]:
len(ibra['sector_aux'].unique())

In [ ]:
len(ibra['sub_sector_aux'].unique())

## Verificando empresas que divergem nos dois índices - Obsoleto

In [ ]:
emp_ibrx = ibrx['codigo']
set_ibrx = set(emp_ibrx)
len(set_ibrx)

In [ ]:
emp_ibov = ibov['codigo']
set_ibov = set(emp_ibov)
len(set_ibov)

In [ ]:
emp_ibra = ibra['codigo']
set_ibra = set(emp_ibra)
len(set_ibra)

In [ ]:
print(len(set_ibra.difference(set_ibrx)))
print(set_ibra.difference(set_ibrx))

In [ ]:
print(len(set_ibov.difference(set_ibrx)))
print(set_ibov.difference(set_ibrx))

In [ ]:
print(len(set_ibrx.difference(set_ibov)))
print(set_ibrx.difference(set_ibov))

### Verificando qual empresa da lista completa não exista em 2021

In [ ]:
set_codigo = set(lista_codigo)
set_empresas_2021 = set(empresas_2021)

print(len(set_codigo.difference(set_empresas_2021)))
print(set_codigo.difference(set_empresas_2021))

## Criando DataFrame unindo os dois índices - Obsoleto

In [ ]:
df_empresas = pd.concat([ibrx, ibov]).drop_duplicates(subset='codigo', ignore_index=True).drop(['qtde_teorica',r'part_%',r'part_%acum'], axis=1)

# Gathering Data - Brazilian Companies Historical Dataset

### Create data set concatenating historical datasets downloaded from B3 website with data from the companies listed in IBRA Index

## Criando lista com o código dos pricipais ativos

In [ ]:
## Utilizar lista do IBRA
codigo = list(set_ibra)

## Unzip files

In [ ]:
import zipfile
path = r'C:\Users\Pedro\OneDrive\Desktop\Ironhack\04. GitHub\data_visualization\data/'

i = 1986

while i < 2000:
    with zipfile.ZipFile(f'{path}COTAHIST_A{i}.zip', 'r') as zip_ref:
        zip_ref.extractall(path)
    i +=1



## Rename files

In [ ]:
os.listdir()

In [ ]:
i = 2000

while i < 2022:
    os.rename(f'COTAHIST_A{i}.TXT', f'COTAHIST_A{i}.txt')
    i +=1

## Setting configurations to read B3 historical files

In [ ]:
pd.set_option('display.max_columns', 500)
pd.options.display.max_columns=500

In [ ]:
path = r'C:\Users\Pedro\OneDrive\Desktop\Ironhack\05. Dados\B3\txt/'

year = 2022

widths = [2,8,2,12,3,12,10,3,4,13,13,13,13,13,13,13,5,18,18,13,1,8,7,13,12,3]

col_names = [
"tipo_registro",
"data_pregao",
"cod_bdi",
"cod_negociacao",
"tipo_mercado",
"nome_empresa",
"especificacao_papel",
"prazo_dias_merc_termo",
"moeda_referencia",
"preco_abertura",
"preco_maximo",
"preco_minimo",
"preco_medio",
"preco_ultimo_negocio",
"preco_melhor_oferta_compra",
"preco_melhor_oferta_venda",
"numero_negocios",
"quantidade_papeis_negociados",
"volume_total_negociado",
"preco_exercicio",
"ìndicador_correcao_precos",
"data_vencimento" ,
"fator_cotacao",
"preco_exercicio_pontos",
"codigo_isin",
"num_distribuicao_papel"]

decimal_config=[
"preco_abertura",
"preco_maximo",
"preco_minimo",
"preco_medio",
"preco_ultimo_negocio",
"preco_melhor_oferta_compra",
"preco_melhor_oferta_venda",
"volume_total_negociado",
"preco_exercicio",
"preco_exercicio_pontos"
]


remains = [
"data_pregao",
"cod_negociacao",
"tipo_mercado",
"nome_empresa",
"preco_abertura",
"preco_maximo",
"preco_minimo",
"preco_medio",
"preco_ultimo_negocio",
]

In [ ]:
dataset = pd.DataFrame()

In [ ]:
'''     Concatenate DataFrames
year = 2022
while year > 1985:
    df = pd.read_fwf(f'{path}COTAHIST_A{year}.TXT',
                                    encoding='ISO-8859-1',
                                    header=0,
                                    widths=widths,
                                    skipfooter=1,
                                    engine='python',
                                    parse_dates=[1],
                                    infer_datetime_format=True,
                                    index_col=False)
    year -= 1

    #Definindo nomes das colunas
    df.columns = col_names

    #Corrigindo casas decimais
    for col in decimal_config:
        df[col]=df[col]/100

    #Selecionando colunas
    df = df[remains]
    
    #Mascara de empresas desejadas
    mask_empresas = df.cod_negociacao.isin(codigo)

    df = df[mask_empresas]

    dataset = pd.concat([dataset, df], ignore_index=True)
'''

## Export DF - Historical Series

In [ ]:
dataset.to_csv(f'{path}dataset_IBRA.csv',
                    encoding='ISO-8859-1',
                    sep=';',
                    decimal='.',
                    index=False)

In [ ]:
pd.read_csv(f'{path}dataset_IBRA.csv',
                encoding='ISO-8859-1',
                sep=';',
                decimal='.',
                index_col=False
                )

# Gathering Data From Yahoo Finance

### Get historical series of brazilian and american indexes
### Get historical series of Gold, Bitcoin and Ethererum
### Get historical series of american companies

## Yahoo Finance

In [ ]:
# Set YFinance setting

symbol_list_br = ['^BVSP','^IBX50']
symbol_list_eua = ['^DJI','^IXIC','^GSPC','GC=F']
symbol_list_crypto = ['BTC-USD', 'ETH-USD']

name_dict={
    '^BVSP':'Ibovespa',
    '^IBX50':'IBrX50',
    '^DJI':'Dow Jones',
    '^IXIC':'NASDAQ',
    '^GSPC':'S&P 500',
    'GC=F':'Ouro ($)',
    'BTC-USD':'Bitcoin ($)',
    'ETH-USD':'Ethereum ($)'
}

col_names = ['date',
"open",
"high",
"low",
"close",
'cod_yfinance']

drop_col = ['Volume','Dividends','Stock Splits']

start = '2000-01-01'
start_br ='2004-11-03'

In [ ]:
# Create empty indexes DataFrames
'''
df_br = pd.DataFrame()
df_eua = pd.DataFrame()
df_crypto = pd.DataFrame()
'''

In [ ]:
# Fill brazilian indexes historical series dataset
'''
for ativo in symbol_list_br:
        chamada_api = yf.Ticker(ativo).history(period='max')
        chamada_api['cod_yfinace'] = ativo
        df_br = pd.concat([df_br, chamada_api])
'''

In [ ]:
# Tranform data from index to column 0
'''
df_br = df_br.drop(drop_col, axis = 1)
df_br.reset_index(inplace=True)
df_br['Date'] = df_br['Date'].dt.date
df_br.columns = col_names
df_br
'''

In [ ]:
# Put indexes names in the DF
df_br['name'] = df_br['cod_yfinance'].map(name_dict)

In [ ]:
# Reroder columns
df_br = df_br[['date','name','open','high','low','close','cod_yfinance']]

In [ ]:
# Save DataFrame
df_br.to_csv(f'{path}index_br.csv',
                    encoding='ISO-8859-1',
                    sep=';',
                    decimal='.',
                    index=False)

In [ ]:
# Read DataFrame
index_br = pd.read_csv(f'{path}index_br.csv',
                    sep = ';',
                    decimal = '.',
                    encoding='ISO-8859-1',
                    index_col=False
)

In [ ]:
index_br

In [ ]:
# Fill american indexes and gold historical series dataset
'''
for ativo in symbol_list_eua:
        chamada_api = yf.Ticker(ativo).history(period='max')
        chamada_api['cod_yfinance'] = ativo
        df_eua = pd.concat([df_eua, chamada_api])
'''

In [ ]:
# Tranform data from index to column 0
'''
df_eua = df_eua.drop(drop_col, axis = 1)
df_eua.reset_index(inplace=True)
df_eua['Date'] = df_eua['Date'].dt.date
df_eua.columns = col_names
df_eua
'''

In [ ]:
# Put indexes names in the DF
df_eua['name'] = df_eua['cod_yfinance'].map(name_dict)

In [ ]:
# Reorder columns
df_eua = df_eua[['date','name','open','high','low','close','cod_yfinance']]

In [ ]:
# Save DataFrame
df_eua.to_csv(f'{path}index_eua.csv',
                    encoding='ISO-8859-1',
                    sep=';',
                    decimal='.',
                    index=False)

In [ ]:
# Read DataFrame
index_eua = pd.read_csv(f'{path}index_eua.csv',
                    sep = ';',
                    decimal = '.',
                    encoding='ISO-8859-1',
                    index_col=False)

In [ ]:
index_eua

In [ ]:
# Fill cripto historical series dataset
'''
for ativo in symbol_list_crypto:
        chamada_api = yf.Ticker(ativo).history(period='max')
        chamada_api['cod_yfinace'] = ativo
        df_crypto = pd.concat([df_crypto, chamada_api])
'''

In [ ]:
# Tranform data from index to column 0
'''
df_crypto = df_crypto.drop(drop_col, axis = 1)
df_crypto.reset_index(inplace=True)
df_crypto['Date'] = df_crypto['Date'].dt.date
df_crypto.columns = col_names
df_crypto
'''

In [ ]:
# Put indexes names in the DF
df_crypto['name'] = df_crypto['cod_yfinance'].map(name_dict)

In [ ]:
# Reorder columns
df_crypto = df_crypto[['date','name','open','high','low','close','cod_yfinance']]

In [ ]:
df_crypto.to_csv(f'{path}crypto.csv',
                    encoding='ISO-8859-1',
                    sep=';',
                    decimal='.',
                    index=False)

In [ ]:
crypto = pd.read_csv(f'{path}crypto.csv',
                    sep = ';',
                    decimal = '.',
                    encoding='ISO-8859-1',
                    index_col=False)


In [ ]:
crypto

# Gathering Data - American Companies

### List of american companies and composition of indexes - S&P500, Dow Jones, Nasdaq

## S&P500

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

data = pd.read_html(url)

In [ ]:
data[0].head()

In [ ]:
sp500 = data[0].iloc[:,[0,1,3,4]]

In [ ]:
sp500.columns = ['ticker', 'name', 'sector', 'sub_industry']

In [ ]:
sp500

In [ ]:
sp500.to_csv(f'{path}SP500.csv',
                    encoding='UTF-8',
                    sep=';',
                    decimal='.',
                    index=False)

In [ ]:
pd.read_csv(f'{path}SP500.csv',
                    encoding='UTF-8',
                    sep=';',
                    decimal='.')

## Nasdaq

In [ ]:
url = 'https://en.wikipedia.org/wiki/Nasdaq-100'

data = pd.read_html(url)

In [ ]:
data[4]
nasdaq = data[4]
nasdaq

In [ ]:
nasdaq.columns = ['name','ticker', 'sector', 'sub_industry']

In [ ]:
nasdaq.to_csv(f'{path}NASDAQ.csv',
                    encoding='UTF-8',
                    sep=';',
                    decimal='.',
                    index=False)

In [ ]:
nasdaq = pd.read_csv(f'{path}NASDAQ.csv',
                    encoding='UTF-8',
                    sep=';',
                    decimal='.')

## Dow Jones

In [ ]:
url = 'https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average'

data = pd.read_html(url)

In [ ]:
data[1]
dow_jones = data[1].iloc[:,[0,2,3]]
dow_jones

In [ ]:
dow_jones.columns = ['name','ticker', 'sector']

In [ ]:
dow_jones.to_csv(f'{path}DOW_JONES.csv',
                    encoding='UTF-8',
                    sep=';',
                    decimal='.',
                    index=False)

In [ ]:
dow_jones = pd.read_csv(f'{path}DOW_JONES.csv',
                    encoding='UTF-8',
                    sep=';',
                    decimal='.')

## Create DataFrame with all companies in american indexes

In [49]:
emp_sp500 = sp500['ticker']
set_sp500 = set(emp_sp500)
len(set_sp500)

503

In [50]:
emp_nasdaq = nasdaq['ticker']
set_nasdaq = set(emp_nasdaq)
len(set_nasdaq)

102

In [51]:
# Companies in Nasdaq that aren't in sp500
print(len(set_nasdaq.difference(set_sp500)))
print(set_nasdaq.difference(set_sp500))
list_nasdaq = list(set_nasdaq.difference(set_sp500))
list_nasdaq

22
{'DOCU', 'ABNB', 'ASML', 'TEAM', 'AZN', 'DDOG', 'WDAY', 'BIDU', 'JD', 'ZM', 'PANW', 'LULU', 'MRVL', 'SGEN', 'PDD', 'LCID', 'CRWD', 'SPLK', 'ZS', 'SIRI', 'MELI', 'NTES'}


['DOCU',
 'ABNB',
 'ASML',
 'TEAM',
 'AZN',
 'DDOG',
 'WDAY',
 'BIDU',
 'JD',
 'ZM',
 'PANW',
 'LULU',
 'MRVL',
 'SGEN',
 'PDD',
 'LCID',
 'CRWD',
 'SPLK',
 'ZS',
 'SIRI',
 'MELI',
 'NTES']

In [52]:
# Create DF with all companies in both indexes
sp500_concat = pd.concat([sp500,nasdaq.loc[nasdaq['ticker'].isin(list_nasdaq)]], axis=0, ignore_index=True)

In [53]:
sp500_concat

,ticker,name,sector,sub_industry
0,MMM,3M,NaN,NaN
1,AOS,A. O. Smith,NaN,NaN
2,ABT,Abbott,NaN,NaN
3,ABBV,AbbVie,NaN,NaN
4,ABMD,Abiomed,NaN,NaN
...,...,...,...,...
520,SIRI,Sirius XM,NaN,NaN
521,SPLK,Splunk,NaN,NaN
522,WDAY,"Workday, Inc.",NaN,NaN
523,ZM,Zoom Video Communications,NaN,NaN


In [54]:
# Save DF
sp500_concat.to_csv(f'{path}eua_all_companies.csv',
                    encoding='UTF-8',
                    sep=';',
                    decimal='.',
                    index=False)

# Gathering Data - Company's Sector and Industry

### Get sector and sub-sector of all companies using WebScrapping on Yahoo Finance website

In [ ]:
ibra = pd.read_csv(f'{path}IBRA.csv',
                    encoding='ISO-8859-1',
                    sep=';',
                    decimal='.')

ibra = ibra[['ticker','name','sector']]
ibra[['sector','sub_industry']] = np.nan

In [55]:
ibra

,ticker,name,sector,sub_industry
0,AERI3,AERIS,Industrials,Specialty Industrial Machinery
1,ARML3,ARMAC,Industrials,Rental & Leasing Services
2,KEPL3,KEPLER WEBER,Industrials,Farm & Heavy Construction Machinery
3,MILS3,MILLS,Industrials,Rental & Leasing Services
4,ROMI3,ROMI,Industrials,Industrial Distribution
...,...,...,...,...
196,LIGT3,LIGHT S/A,Utilities,Utilities—Renewable
197,NEOE3,NEOENERGIA,Utilities,Utilities—Regulated Electric
198,MEGA3,OMEGAENERGIA,Utilities,Utilities—Renewable
199,TAEE11,TAESA,Utilities,Utilities—Regulated Electric


In [56]:
eua = pd.read_csv(f'{path}eua_all_companies.csv',
                    encoding='UTF-8',
                    sep=';',
                    decimal='.')

In [60]:
eua

,ticker,name,sector,sub_industry
0,MMM,3M,NaN,NaN
1,AOS,A. O. Smith,NaN,NaN
2,ABT,Abbott,NaN,NaN
3,ABBV,AbbVie,NaN,NaN
4,ABMD,Abiomed,NaN,NaN
...,...,...,...,...
520,SIRI,Sirius XM,NaN,NaN
521,SPLK,Splunk,NaN,NaN
522,WDAY,"Workday, Inc.",NaN,NaN
523,ZM,Zoom Video Communications,NaN,NaN


In [59]:
eua.isnull().sum()

ticker            0
name              0
sector          525
sub_industry    525
dtype: int64

In [58]:
eua[eua['ticker'].str.contains(r'[^a-zA-Z0-9]')]

,ticker,name,sector,sub_industry
65,BRK-B,Berkshire Hathaway,NaN,NaN
81,BF-B,Brown–Forman,NaN,NaN


In [17]:
# Replace '.' in S&P500 ticker to '-', in order to get the right ticker for yfinance webscrapping
eua['ticker'].replace(r'[^a-zA-Z0-9]', r'-', regex=True, inplace=True)

In [22]:
#Runs the webdriver
driver = webdriver.Chrome(options=chromeOptions)

In [61]:
index = 0
wait = 8

for i in eua['ticker']:
    # Get the Yfinance company's url
    url = f'https://finance.yahoo.com/quote/{i}/profile?p={i}'
    driver.get(url)
    driver.implicitly_wait(wait)

    # Get company's sector
    try:
        sector = driver.find_element(By.XPATH, '//*[@id="Col1-0-Profile-Proxy"]/section/div[1]/div/div/p[2]/span[2]')
        driver.implicitly_wait(wait)
        eua.loc[index, 'sector'] = sector.text
    except:
        pass

    # Get company's industry
    try:
        industry = driver.find_element(By.XPATH, '//*[@id="Col1-0-Profile-Proxy"]/section/div[1]/div/div/p[2]/span[4]')
        driver.implicitly_wait(wait)
        eua.loc[index, 'sub_industry'] = industry.text
    except:
        pass

    # Update index
    index +=1    

### Checking if all companies in IBRA index have sector and industry

In [47]:
ibra[ibra['sector'].isna()]

,ticker,name,sector,sub_industry


In [89]:
ibra['sector'].unique()

array(['Industrials', 'Consumer Cyclical', 'Technology',
       'Consumer Defensive', 'Basic Materials', 'Utilities',
       'Real Estate', 'Financial Services', 'Energy', 'Healthcare',
       'Communication Services'], dtype=object)

In [83]:
ibra[ibra['sector'] == '']

,ticker,name,sector,sub_industry
149,BLAU3,BLAU,,
151,HYPE3,HYPERA,,
200,TRPL4,TRAN PAULIST,,


In [86]:
ibra.loc[[149,151,200]]

,ticker,name,sector,sub_industry
149,BLAU3,BLAU,Healthcare,Drug Manufacturers—Specialty & Generic
151,HYPE3,HYPERA,Healthcare,Drug Manufacturers—Specialty & Generic
200,TRPL4,TRAN PAULIST,Utilities,Utilities—Independent Power Producers


In [85]:
ibra.loc[149,('sector','sub_industry')] = ['Healthcare', 'Drug Manufacturers—Specialty & Generic']
ibra.loc[151,('sector','sub_industry')] = ['Healthcare', 'Drug Manufacturers—Specialty & Generic']
ibra.loc[200,('sector','sub_industry')] = ['Utilities', 'Utilities—Independent Power Producers']

C:\Users\Pedro\AppData\Local\Temp\ipykernel_3584\3496707366.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ibra.loc[149,('sector','sub_industry')] = ['Healthcare', 'Drug Manufacturers—Specialty & Generic']
C:\Users\Pedro\AppData\Local\Temp\ipykernel_3584\3496707366.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ibra.loc[151,('sector','sub_industry')] = ['Healthcare', 'Drug Manufacturers—Specialty & Generic']
C:\Users\Pedro\AppData\Local\Temp\ipykernel_3584\3496707366.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://

In [87]:
ibra.to_csv(f'{path}IBRA_sector.csv',
                    encoding='UTF-8',
                    sep=';',
                    decimal='.',
                    index=False)

In [88]:
ibra = ibra.rename({'sub_industry':'industry'}, axis=1)

In [91]:
ibra

,ticker,name,sector,industry
0,AERI3,AERIS,Industrials,Specialty Industrial Machinery
1,ARML3,ARMAC,Industrials,Rental & Leasing Services
2,KEPL3,KEPLER WEBER,Industrials,Farm & Heavy Construction Machinery
3,MILS3,MILLS,Industrials,Rental & Leasing Services
4,ROMI3,ROMI,Industrials,Industrial Distribution
...,...,...,...,...
196,LIGT3,LIGHT S/A,Utilities,Utilities—Renewable
197,NEOE3,NEOENERGIA,Utilities,Utilities—Regulated Electric
198,MEGA3,OMEGAENERGIA,Utilities,Utilities—Renewable
199,TAEE11,TAESA,Utilities,Utilities—Regulated Electric


In [110]:
cod_yfinance = [f'{i}.SA' for i in ibra['ticker']]

In [111]:
ibra['cod_yfinance'] = cod_yfinance

In [113]:
ibra['country'] = 'Brazil'

In [114]:
ibra

,ticker,name,sector,industry,cod_yfinance,country
0,AERI3,AERIS,Industrials,Specialty Industrial Machinery,AERI3.SA,Brazil
1,ARML3,ARMAC,Industrials,Rental & Leasing Services,ARML3.SA,Brazil
2,KEPL3,KEPLER WEBER,Industrials,Farm & Heavy Construction Machinery,KEPL3.SA,Brazil
3,MILS3,MILLS,Industrials,Rental & Leasing Services,MILS3.SA,Brazil
4,ROMI3,ROMI,Industrials,Industrial Distribution,ROMI3.SA,Brazil
...,...,...,...,...,...,...
196,LIGT3,LIGHT S/A,Utilities,Utilities—Renewable,LIGT3.SA,Brazil
197,NEOE3,NEOENERGIA,Utilities,Utilities—Regulated Electric,NEOE3.SA,Brazil
198,MEGA3,OMEGAENERGIA,Utilities,Utilities—Renewable,MEGA3.SA,Brazil
199,TAEE11,TAESA,Utilities,Utilities—Regulated Electric,TAEE11.SA,Brazil


### Checking if all companies in eua DataFrame have sector and industry

In [70]:
eua.isnull().sum()

ticker          0
name            0
sector          0
sub_industry    0
dtype: int64

In [90]:
eua['sector'].unique()

array(['Industrials', 'Healthcare', 'Technology',
       'Communication Services', 'Consumer Defensive',
       'Consumer Cyclical', 'Utilities', 'Financial Services',
       'Basic Materials', 'Real Estate', 'Energy'], dtype=object)

In [63]:
# Save Dataframe
eua.to_csv(f'{path}eua_all_companies.csv',
                    encoding='UTF-8',
                    sep=';',
                    decimal='.',
                    index=False)

In [75]:
eua = eua.rename({'sub_industry':'industry'}, axis = 1)

In [115]:
eua['cod_yfinance'] = eua['ticker']

In [117]:
eua['country'] = 'USA'

In [118]:
eua

,ticker,name,sector,industry,cod_yfinance,country
0,MMM,3M,Industrials,Conglomerates,MMM,USA
1,AOS,A. O. Smith,Industrials,Specialty Industrial Machinery,AOS,USA
2,ABT,Abbott,Healthcare,Medical Devices,ABT,USA
3,ABBV,AbbVie,Healthcare,Drug Manufacturers—General,ABBV,USA
4,ABMD,Abiomed,Healthcare,Medical Devices,ABMD,USA
...,...,...,...,...,...,...
520,SIRI,Sirius XM,Communication Services,Entertainment,SIRI,USA
521,SPLK,Splunk,Technology,Software—Infrastructure,SPLK,USA
522,WDAY,"Workday, Inc.",Technology,Software—Application,WDAY,USA
523,ZM,Zoom Video Communications,Technology,Software—Application,ZM,USA


# Updating IBRA dataset - Obsoleto - criar novo dataframe com series históricas retiradas do yfinace para todas as empresas

### (dataset that contains brazilian companies's historical data)

# Create Datasets

### Dataset that contains brazilian and american companies's characteristics
### Dataset that contains brazilian and american companies's historical data

In [150]:
# Historical dataset is to big to upload to GitHub
stock_path = r'C:\Users\Pedro\OneDrive\Desktop\Ironhack\05. Dados\stock_project_datasets/'

## Dataset with all companies's characteristics

In [126]:
ibra.head()

,ticker,name,sector,industry,cod_yfinance,country
0,AERI3,AERIS,Industrials,Specialty Industrial Machinery,AERI3.SA,Brazil
1,ARML3,ARMAC,Industrials,Rental & Leasing Services,ARML3.SA,Brazil
2,KEPL3,KEPLER WEBER,Industrials,Farm & Heavy Construction Machinery,KEPL3.SA,Brazil
3,MILS3,MILLS,Industrials,Rental & Leasing Services,MILS3.SA,Brazil
4,ROMI3,ROMI,Industrials,Industrial Distribution,ROMI3.SA,Brazil


In [127]:
eua.head()

,ticker,name,sector,industry,cod_yfinance,country
0,MMM,3M,Industrials,Conglomerates,MMM,USA
1,AOS,A. O. Smith,Industrials,Specialty Industrial Machinery,AOS,USA
2,ABT,Abbott,Healthcare,Medical Devices,ABT,USA
3,ABBV,AbbVie,Healthcare,Drug Manufacturers—General,ABBV,USA
4,ABMD,Abiomed,Healthcare,Medical Devices,ABMD,USA


In [153]:
all_companies = pd.concat([ibra, eua], axis=0, ignore_index=True)

In [157]:
all_companies.to_csv(f'{stock_path}all_companies.csv',
                    encoding='UTF-8',
                    sep=';',
                    decimal='.',
                    index=False)

## Dataset with all companies historical data

In [147]:
# Create empty DataFrame
historical_data = pd.DataFrame()

In [148]:
# Creating DataFrame with historical data of all companies
for ticker in eua['cod_yfinance']:
    aux_df = pd.DataFrame()
    aux_df = yf.Ticker(ticker).history(period='max')
    aux_df['cod_yfinance'] = ticker

    historical_data = pd.concat([historical_data, aux_df], axis=0)

for ticker in ibra['cod_yfinance']:
    aux_df = pd.DataFrame()
    aux_df = yf.Ticker(ticker).history(period='max')
    aux_df['cod_yfinance'] = ticker

    historical_data = pd.concat([historical_data, aux_df], axis=0)

Failed to get ticker 'MRFG3.SA' reason: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Max retries exceeded with url: /v8/finance/chart/MRFG3.SA?range=1d&interval=1d (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002209B993490>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
- MRFG3.SA: No data found for this date range, symbol may be delisted
Failed to get ticker 'CYRE3.SA' reason: ('Connection aborted.', TimeoutError(10060, 'Uma tentativa de conexão falhou porque o componente conectado não respondeu\r\ncorretamente após um período de tempo ou a conexão estabelecida falhou\r\nporque o host conectado não respondeu', None, 10060, None))
- CYRE3.SA: No data found for this date range, symbol may be delisted


In [152]:
# Check if got all companies historical data
len(historical_data['cod_yfinance'].unique())

724

In [170]:
# Drop unecessary column
historical_data.drop('Adj Close', axis=1, inplace=True)

In [164]:
# Getting companies that failed previously
for ticker in ['MRFG3.SA','CYRE3.SA']:
    aux_df = pd.DataFrame()
    aux_df = yf.Ticker(ticker).history(period='max')
    aux_df['cod_yfinance'] = ticker

    historical_data = pd.concat([historical_data, aux_df], axis=0)

In [172]:
# Save DataFrame
historical_data.to_csv(f'{stock_path}historical_data_rev1.csv',
                    encoding='UTF-8',
                    sep=';',
                    decimal='.',
                    index=False)

In [171]:
historical_data

,Open,High,Low,Close,Volume,Dividends,Stock Splits,cod_yfinance
Date,,,,,,,,
1962-01-02 00:00:00-05:00,0.00,0.762121,0.739706,0.745309,212800.0,0.0,0.0,MMM
1962-01-03 00:00:00-05:00,0.00,0.750913,0.732701,0.750913,422400.0,0.0,0.0,MMM
1962-01-04 00:00:00-05:00,0.00,0.763522,0.750913,0.750913,212800.0,0.0,0.0,MMM
1962-01-05 00:00:00-05:00,0.00,0.748110,0.728497,0.731299,315200.0,0.0,0.0,MMM
1962-01-08 00:00:00-05:00,0.00,0.732700,0.722893,0.727096,334400.0,0.0,0.0,MMM
...,...,...,...,...,...,...,...,...
2022-12-05 00:00:00-03:00,14.45,14.470000,13.470000,13.610000,9161100.0,0.0,0.0,CYRE3.SA
2022-12-06 00:00:00-03:00,13.66,13.870000,13.520000,13.740000,6349200.0,0.0,0.0,CYRE3.SA
2022-12-07 00:00:00-03:00,13.69,14.150000,13.690000,13.860000,5321500.0,0.0,0.0,CYRE3.SA


# Working on companies sector and industry

In [173]:
all_companies['sector'].value_counts()

Industrials               104
Consumer Cyclical          98
Technology                 93
Financial Services         93
Healthcare                 81
Consumer Defensive         54
Utilities                  54
Real Estate                47
Basic Materials            37
Communication Services     33
Energy                     32
Name: sector, dtype: int64

In [178]:
all_companies[all_companies['ticker'].str.contains('ABC')]

,ticker,name,sector,industry,cod_yfinance,country
97,ABCB4,ABC BRASIL,Financial Services,Banks—Regional,ABCB4.SA,Brazil
238,ABC,AmerisourceBergen,Healthcare,Medical Distribution,ABC,USA


In [181]:
all_companies['industry']

0           Specialty Industrial Machinery
1                Rental & Leasing Services
2      Farm & Heavy Construction Machinery
3                Rental & Leasing Services
4                  Industrial Distribution
                      ...                 
721                          Entertainment
722                Software—Infrastructure
723                   Software—Application
724                   Software—Application
725                Software—Infrastructure
Name: industry, Length: 726, dtype: object

In [190]:
all_companies['industry'] = all_companies['industry'].str.replace("—"," - ")

In [192]:
all_companies[all_companies['industry'].str.contains('-')]

,ticker,name,sector,industry,cod_yfinance,country
36,ABEV3,AMBEV S/A,Consumer Defensive,Beverages - Brewers,ABEV3.SA,Brazil
44,RAIZ4,RAIZEN,Utilities,Utilities - Renewable,RAIZ4.SA,Brazil
66,CURY3,CURY S/A,Real Estate,Real Estate - Development,CURY3.SA,Brazil
68,DIRR3,DIRECIONAL,Real Estate,Real Estate - Development,DIRR3.SA,Brazil
69,EVEN3,EVEN,Real Estate,Real Estate - Development,EVEN3.SA,Brazil
...,...,...,...,...,...,...
718,PANW,Palo Alto Networks,Technology,Software - Infrastructure,PANW,USA
722,SPLK,Splunk,Technology,Software - Infrastructure,SPLK,USA
723,WDAY,"Workday, Inc.",Technology,Software - Application,WDAY,USA
724,ZM,Zoom Video Communications,Technology,Software - Application,ZM,USA


In [193]:
all_companies.to_csv(f'{stock_path}all_companies_rev2.csv',
                    encoding='UTF-8',
                    sep=';',
                    decimal='.',
                    index=False)

In [199]:
agf = ['QUAL3'
,'VVBR3'
,'AESB3'
,'WIZS3'
,'BRAP3'
,'BRSR6'
,'BRKM5'
,'BRAP4'
,'BRKM3'
,'BRSR3'
,'CGAS5'
,'TRPL4'
,'VIVT3'
,'TAEE11'
,'TAEE4'
,'TAEE3'
,'CGAS3'
,'ITSA4'
,'CSMG3'
,'ENAT3'
,'ITSA3'
,'SANB11'
,'SANB3'
,'SANB4'
,'GRND3'
,'BRSR5'
,'TRPL3'
,'SAPR4'
,'SAPR3'
,'BBSE3'
,'CMIG4'
,'ALUP11'
,'CLSC3'
,'CMIG4'
,'ELET3'
,'BBAS3'
,'BBDC4'
,'ITUB4'
,'AMBP3'
,'CSAN3'
,'CSAN4'
,'OPCT3'
,'SBSP3'
,'SAPR11'
,'VERZ34'
,'OIBR4'
,'TIMS3'
,'VIVT4'
,'TELB4'
,'TELB3'
,'ATTB34'
]


In [200]:
all_companies[all_companies['ticker'].isin(agf)]

,ticker,name,sector,industry,cod_yfinance,country
60,GRND3,GRENDENE,Consumer Cyclical,Footwear & Accessories,GRND3.SA,Brazil
99,BRSR6,BANRISUL,Financial Services,Banks - Regional,BRSR6.SA,Brazil
101,BBDC4,BRADESCO,Financial Services,Banks - Regional,BBDC4.SA,Brazil
102,BBAS3,BRASIL,Financial Services,Banks - Regional,BBAS3.SA,Brazil
104,ITSA4,ITAUSA,Financial Services,Banks - Regional,ITSA4.SA,Brazil
106,ITUB4,ITAUUNIBANCO,Financial Services,Banks - Regional,ITUB4.SA,Brazil
108,SANB11,SANTANDER BR,Financial Services,Banks - Regional,SANB11.SA,Brazil
109,BBSE3,BBSEGURIDADE,Financial Services,Insurance - Diversified,BBSE3.SA,Brazil
114,WIZS3,WIZ S.A.,Financial Services,Insurance Brokers,WIZS3.SA,Brazil
124,BRAP3,BRADESPAR,Financial Services,Asset Management,BRAP3.SA,Brazil


In [222]:
all_companies.loc[all_companies['name'].isin(companies), 'industry'].unique()

array(['Auto Parts', 'Infrastructure Operations', 'Farm Products',
       'Footwear & Accessories', 'Banks - Regional', 'Capital Markets',
       'Insurance - Diversified', 'Insurance - Life', 'Insurance Brokers',
       'Financial Data & Stock Exchanges', 'Paper & Paper Products',
       'Asset Management', 'Other Industrial Metals & Mining',
       'Chemicals', 'Steel', 'Oil & Gas Refining & Marketing',
       'Oil & Gas E&P', 'Drug Manufacturers - Specialty & Generic',
       'Diagnostics & Research', 'Healthcare Plans', 'Telecom Services',
       'Utilities - Regulated Water', 'Utilities - Renewable',
       'Utilities - Diversified', 'Utilities - Regulated Electric',
       'Utilities - Independent Power Producers'], dtype=object)

In [224]:
all_companies.loc[all_companies['name'].isin(companies)]

,ticker,name,sector,industry,cod_yfinance,country
11,TUPY3,TUPY,Consumer Cyclical,Auto Parts,TUPY3.SA,Brazil
16,CCRO3,CCR SA,Industrials,Infrastructure Operations,CCRO3.SA,Brazil
45,SLCE3,SLC AGRICOLA,Consumer Defensive,Farm Products,SLCE3.SA,Brazil
60,GRND3,GRENDENE,Consumer Cyclical,Footwear & Accessories,GRND3.SA,Brazil
64,MYPK3,IOCHP-MAXION,Consumer Cyclical,Auto Parts,MYPK3.SA,Brazil
65,LEVE3,METAL LEVE,Consumer Cyclical,Auto Parts,LEVE3.SA,Brazil
97,ABCB4,ABC BRASIL,Financial Services,Banks - Regional,ABCB4.SA,Brazil
99,BRSR6,BANRISUL,Financial Services,Banks - Regional,BRSR6.SA,Brazil
100,BBDC3,BRADESCO,Financial Services,Banks - Regional,BBDC3.SA,Brazil
101,BBDC4,BRADESCO,Financial Services,Banks - Regional,BBDC4.SA,Brazil


In [225]:
perennial=[
'Utilities - Diversified'
,'Utilities - Independent Power Producers'
,'Utilities - Regulated Electric'
,'Utilities - Regulated Gas'
,'Utilities - Regulated Water'
,'Utilities - Renewable'
,'Banks - Diversified'
,'Banks - Regional'
,'Insurance - Diversified'
,'Insurance - Life'
,'Insurance - Property & Casualty'
,'Insurance - Reinsurance'
,'Insurance - Specialty'
,'Insurance Brokers'
,'Insurance Brokers'
,'Healthcare Plans'
,'Telecom Services'
,'Waste Management'
,'Oil & Gas E&P'
,'Oil & Gas Equipment & Services'
,'Oil & Gas Integrated'
,'Oil & Gas Midstream'
,'Oil & Gas Refining & Marketing']

In [226]:
all_companies.loc[all_companies['industry'].isin(perennial)]

,ticker,name,sector,industry,cod_yfinance,country
44,RAIZ4,RAIZEN,Utilities,Utilities - Renewable,RAIZ4.SA,Brazil
97,ABCB4,ABC BRASIL,Financial Services,Banks - Regional,ABCB4.SA,Brazil
98,BPAN4,BANCO PAN,Financial Services,Banks - Regional,BPAN4.SA,Brazil
99,BRSR6,BANRISUL,Financial Services,Banks - Regional,BRSR6.SA,Brazil
100,BBDC3,BRADESCO,Financial Services,Banks - Regional,BBDC3.SA,Brazil
...,...,...,...,...,...,...
686,WFC,Wells Fargo,Financial Services,Banks - Diversified,WFC,USA
693,WMB,Williams Companies,Energy,Oil & Gas Midstream,WMB,USA
694,WTW,Willis Towers Watson,Financial Services,Insurance Brokers,WTW,USA
697,XEL,Xcel Energy,Utilities,Utilities - Regulated Electric,XEL,USA


In [201]:
#Runs the webdriver
driver = webdriver.Chrome(options=chromeOptions)

In [ ]:
driver.implicitly_wait(wait)

In [205]:
company_name = driver.find_elements(By.CSS_SELECTOR, '.text-secondary')

In [208]:
companies = [i.text for i in company_name]

In [219]:
br_ticker = driver.find_elements(By.XPATH, '//*[contains(concat( " ", @class, " " ), concat( " ", "text-muted", " " ))]')

In [220]:
[i.text for i in br_ticker]

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'PL',
 'QUAL3',
 'VBBR3',
 'AESB3',
 'WIZS3',
 'BRAP3',
 'BRSR6',
 'BRKM5',
 'BRAP4',
 'BRKM3',
 'BRSR3',
 'CGAS5',
 'TRPL4',
 'VIVT3',
 'TAEE11',
 'TAEE4',
 'TAEE3',
 'CGAS3',
 'ITSA4',
 'CSMG3',
 'ENAT3',
 'ITSA3',
 'SANB11',
 'SANB3',
 'SANB4',
 'GRND3',
 'BRSR5',
 'TRPL3',
 'SAPR4',
 'SAPR3',
 'BBSE3',
 'BBDC3',
 'BRIV4',
 'ITUB3',
 'BBDC4',
 'FLRY3',
 'EGIE3',
 'CPLE3',
 'CPLE6',
 'ITUB4',
 'CCRO3',
 'ROMI3',
 'PSSA3',
 'ABCB4',
 'UGPA3',
 'SAPR11',
 'BBAS3',
 'ODPV3',
 'CMIG4',
 'MYPK3',
 'VALE3',
 'UNIP6',
 'UNIP5',
 'LEVE3',
 'UNIP3',
 'B3SA3',
 'BMEB4',
 'ABEV3',
 'SULA4',
 'SULA11',
 'SULA3',
 'CLSC4',
 'CPFE3',
 'ENGI4',
 'ENBR3',
 'CLSC3',
 'NEOE3',
 'ELET6',
 'TUPY3',
 'KLBN3',
 'KLBN11',
 'KLBN4',
 'ALUP4',
 'ALUP11',
 'ALUP3',
 'HYPE3',
 'CPLE5',
 'ELET5',
 'CSAN3',
 'CMIG3',
 'BPAC5',
 'BMEB3',
 'ENGI3',
 'ELET3',
 'SLCE3